In [9]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import twitter_samples

# TASK 0

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import twitter_samples

tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
tokens = tweet_tokens[50]
lemmatizer = WordNetLemmatizer()

def lemmatize_sentence(tokens):
    lemmatized_sentence = []
    # CODE_START
    for token, tag in pos_tag(tokens):
        # Перетворення тегів PoS
        if tag.startswith('NN'):
            pos = 'n' # Noun
        elif tag.startswith('VB'):
            pos = 'v' # Verb
        else:
            pos = 'a' # Adjective (для решти випадків)
        
        # Лематизація токена з відповідним тегом
        lemmatized_token = lemmatizer.lemmatize(token, pos)
        lemmatized_sentence.append(lemmatized_token)
    # CODE_END
    return lemmatized_sentence

# Тестування функції
print("Оригінальні токени:", tokens)
print("Лематизовані токени:", lemmatize_sentence(tokens))

Оригінальні токени: ['@groovinshawn', 'they', 'are', 'rechargeable', 'and', 'it', 'normally', 'comes', 'with', 'a', 'charger', 'when', 'u', 'buy', 'it', ':)']
Лематизовані токени: ['@groovinshawn', 'they', 'be', 'rechargeable', 'and', 'it', 'normally', 'come', 'with', 'a', 'charger', 'when', 'u', 'buy', 'it', ':)']


In [11]:
import re, string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag

def process_tokens(tweet_tokens):
    cleaned_tokens = []
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()

    for token, tag in pos_tag(tweet_tokens):
        # CODE_START
        # Видалення URL та згадок
        token = re.sub('(@[A-Za-z0-9_]+)|(https?://[A-Za-z0-9./]+)', '', token)

        # Перевірка на стоп-слова та пунктуацію після видалення regex
        if token and token.lower() not in stop_words and token not in string.punctuation:
            # Перетворення тегів PoS
            if tag.startswith('NN'):
                pos = 'n'
            elif tag.startswith('VB'):
                pos = 'v'
            else:
                pos = 'a'

            # Лематизація та приведення до нижнього регістру
            cleaned_token = lemmatizer.lemmatize(token.lower(), pos)
            cleaned_tokens.append(cleaned_token)
        # CODE_END
    return cleaned_tokens

# Тестування функції
print("\nBefore:", tweet_tokens[50])
print("After:", process_tokens(tweet_tokens[50]))


Before: ['@groovinshawn', 'they', 'are', 'rechargeable', 'and', 'it', 'normally', 'comes', 'with', 'a', 'charger', 'when', 'u', 'buy', 'it', ':)']
After: ['rechargeable', 'normally', 'come', 'charger', 'u', 'buy', ':)']


In [12]:
# CODE_START
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = [process_tokens(tokens) for tokens in positive_tweet_tokens]
negative_cleaned_tokens_list = [process_tokens(tokens) for tokens in negative_tweet_tokens]
# CODE_END

# Перевірка результату
print("\nОригінальний твіт 500:", positive_tweet_tokens[500])
print("Очищений твіт 500:", positive_cleaned_tokens_list[500])


Оригінальний твіт 500: ['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
Очищений твіт 500: ['dang', 'rad', '#fanart', ':d']


In [13]:
from nltk import FreqDist

def get_all_words(cleaned_tokens_list):
    # CODE_START
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token
    # CODE_END

all_pos_words = list(get_all_words(positive_cleaned_tokens_list))

# CODE_START
# Виконання частотного аналізу
freq_dist_pos = FreqDist(all_pos_words)
print("\n10 найпоширеніших слів у позитивних твітах:")
print(freq_dist_pos.most_common(10))
# CODE_END


10 найпоширеніших слів у позитивних твітах:
[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 383), ('follow', 362), ('love', 336), ('...', 290), ('good', 283), ('get', 269), ('thank', 258)]


# TASK 1 

In [14]:
def process_tokens_no_hashtags(tweet_tokens):
    cleaned_tokens = []
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()

    for token, tag in pos_tag(tweet_tokens):
        # Додано патерн для хештегів: (#[A-Za-z0-9_]+)
        token = re.sub('(@[A-Za-z0-9_]+)|(https?://[A-Za-z0-9./]+)|(#[A-Za-z0-9_]+)', '', token)

        if token and token.lower() not in stop_words and token not in string.punctuation:
            if tag.startswith('NN'):
                pos = 'n'
            elif tag.startswith('VB'):
                pos = 'v'
            else:
                pos = 'a'
            cleaned_token = lemmatizer.lemmatize(token.lower(), pos)
            cleaned_tokens.append(cleaned_token)
            
    return cleaned_tokens

# Тестовий твіт з хештегом
test_tokens_hashtag = ['I', 'love', '#NLP', 'and', '#AI', '!', 'It', 'is', 'awesome', '.']
print("\nТест видалення хештегів:")
print("Оригінал:", test_tokens_hashtag)
print("Після очищення:", process_tokens_no_hashtags(test_tokens_hashtag))


Тест видалення хештегів:
Оригінал: ['I', 'love', '#NLP', 'and', '#AI', '!', 'It', 'is', 'awesome', '.']
Після очищення: ['love', 'awesome']


# TASK 2

In [15]:
from nltk.corpus import wordnet as wn

def process_tokens_synsets(tweet_tokens):
    cleaned_tokens = []
    stop_words = stopwords.words('english')

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('(@[A-Za-z0-9_]+)|(https?://[A-Za-z0-9./]+)', '', token)

        if token and token.lower() not in stop_words and token not in string.punctuation:
            # Конвертація тега
            if tag.startswith('NN'):
                pos = 'n'
            elif tag.startswith('VB'):
                pos = 'v'
            else:
                pos = 'a'
            
            # Пошук синсетів
            synsets = wn.synsets(token.lower(), pos)
            
            # Якщо синсети знайдено, беремо ім'я першого (найпоширенішого)
            if synsets:
                cleaned_tokens.append(synsets[0].name())
            # Інакше, можемо додати сам токен (опціонально)
            # else:
            #     cleaned_tokens.append(token.lower())
                
    return cleaned_tokens
    
# Тестування функції
print("\nТест заміни на синсети:")
print("Оригінал:", tweet_tokens[50])
print("Після обробки з синсетами:", process_tokens_synsets(tweet_tokens[50]))


Тест заміни на синсети:
Оригінал: ['@groovinshawn', 'they', 'are', 'rechargeable', 'and', 'it', 'normally', 'comes', 'with', 'a', 'charger', 'when', 'u', 'buy', 'it', ':)']
Після обробки з синсетами: ['rechargeable.s.01', 'come.v.01', 'charger.n.01', 'u.s.01', 'buy.v.01']


# TASK 3

In [16]:
def semantic_distance(word1, word2):
    """
    Обчислює семантичну відстань між двома словами.
    Відстань = відстань(слово1 -> спільний_предок) + відстань(слово2 -> спільний_предок)
    """
    # Беремо перший (найпоширеніший) синсет для кожного слова
    syn1 = wn.synsets(word1)
    syn2 = wn.synsets(word2)
    
    if not syn1 or not syn2:
        print(f"Одне зі слів ('{word1}' або '{word2}') не знайдено в WordNet.")
        return None
        
    s1 = syn1[0]
    s2 = syn2[0]

    # Знаходимо найближчий спільний гіперонім
    common_hypernyms = s1.lowest_common_hypernyms(s2)
    
    if not common_hypernyms:
        print(f"Не знайдено спільних гіперонімів для '{word1}' та '{word2}'.")
        return None
        
    common = common_hypernyms[0]
    
    # Обчислюємо відстань від кожного синсета до спільного предка
    dist1 = s1.shortest_path_distance(common)
    dist2 = s2.shortest_path_distance(common)
    
    return dist1 + dist2

# Приклади використання
dist_car_auto = semantic_distance("car", "automobile")
print(f"\nСемантична відстань між 'car' та 'automobile': {dist_car_auto}") # Очікується 0, бо це синоніми в одному синсеті

dist_car_boat = semantic_distance("car", "boat")
print(f"Семантична відстань між 'car' та 'boat': {dist_car_boat}")

dist_tree_cat = semantic_distance("tree", "cat")
print(f"Семантична відстань між 'tree' та 'cat': {dist_tree_cat}")


Семантична відстань між 'car' та 'automobile': 0
Семантична відстань між 'car' та 'boat': 7
Семантична відстань між 'tree' та 'cat': 12
